## Task

Your task in this assignment is to predict the ethnicity of someone's DNA, based on single nucleotide polymorphism data we've shared with you.

## Data values

Each input vector represents the DNA at specific locations in the genome for one individual. There are 20 binary input features. 0 indicates that the user's DNA at the given location matches the human reference genome. 1 indicates that the user's DNA does not match the human reference genome. The output class value represents the super population (ethnicity) of each individual. The super populations contained in this dataset are East Asian or Mixed American, encoded in binary. The training data set contains 283 data vectors, and the testing data set contains 184 data vectors.

We will do this in three different ways. Firstly, (Part A) we will do it visually by exmaining the graphs of the distributions and trying to guess which input features are likely to be relevant in predicting ethnicityThen we write If..Else statements to classify manually. Next (Part B), we replace our visual inspection with a correelation map and use a threshold that can be changed to see the trade-off between FPs and FNs. Lastly (Part C), we will use a pre-existing library for Logistic Regression to demonstrate the power of Python libraries for Machine Learning!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dython.nominal import associations
import csv
import scipy.stats as stats
plt.style.use('fivethirtyeight')

In [ ]:
df = pd.read_csv('ancestry-train.txt', sep= " ", header=None)
df.columns = ['SNP1', 'SNP2', 'SNP3', 'SNP4','SNP5', 'SNP6', 'SNP7', 'SNP8', 'SNP9', 'SNP10', 'SNP11', 'SNP12', 'SNP13', 'SNP14', 'SNP15', 'SNP16', 'SNP17', 'SNP18', 'SNP19', 'SNP20', 'Ethnicity']
df.head()

Fixing the colon glitch for SNP20

In [ ]:
def fix(x):
    x = x[0]
    return int(x)

In [ ]:
df['SNP20'] = df['SNP20'].apply(fix)

## Part A- 
Visual data analysis

In [ ]:
df.groupby('Ethnicity').mean().plot(kind='bar', yerr = 1.96*df.groupby('Ethnicity').std()/np.sqrt(len(df)))
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
df1 = df.groupby('Ethnicity').mean() # This creates two rows, one for each value in the groupby clause. 
# Calling mean() on this groupby object calculates the mean for each numeric column. 
df1

In [ ]:
df1 = df1.diff() #this line gives the difference between each row of the df and the preceding one.
df1.reset_index() #this is needed to create an index based onthe group names (0,1)

In [ ]:
# The point of doing this was to see which of the input features have comparatively more different mean 
# in two class groups. e.g. SNP20 (-0.345592)

In [ ]:
df1.sort_values(by = 1, axis=1, ascending=False, inplace=True)

In [ ]:
pd.crosstab(df.Ethnicity, df.SNP20) # this is a normal crosstab summary

In [ ]:
pd.crosstab(df.Ethnicity, df.SNP20, normalize='index') # and this is a normalised one

In [ ]:
df.groupby('Ethnicity').mean().plot(kind='bar')

## Deciding relevance visually (and also by the diff())
We can notice that SNP18,SNP12,SNP20,SN16 are all more different in two classes.

In [ ]:
df = df.assign(Prediction = "")

In [ ]:
# You will get a SettingWithCopyWarning warning. You can ignore it. It's a warning, not an error.
for (row_index, row_data) in df.iterrows():
    if(df['SNP18'][row_index] == 1):
        df['Prediction'][row_index] = 0
    elif(df['SNP12'][row_index] == 0):
        df['Prediction'][row_index] = 1
    elif((df['SNP12'][row_index] == 1) & (df['SNP20'][row_index] == 0) & (df['SNP16'][row_index] == 0)):
        df['Prediction'][row_index] = 1
    elif((df['SNP12'][row_index] == 1) & (df['SNP20'][row_index] == 1)):
        df['Prediction'][row_index] = 0
    elif(np.random.rand() > 0.5):
        df['Prediction'][row_index] = 0
    else:
        df['Prediction'][row_index] = 1    

## Evaluating model performance

In [ ]:
pd.crosstab(df['Ethnicity'], df['Prediction']) #When you run this, you basically get the Confusion Matrix.
# We get 40 FPs, 17 FNs as errors
# P = 160/200 = 0.8, R = 160/177 = 0.90, F1 =  0.85

## Part B- 
Correlation

In [ ]:
# The Os and 1s are not really numbers. They are binary codes given to classes. Let's get rid of them!

In [ ]:
df["Ethnicity"]=df.apply(lambda row: "East Asian" if row["Ethnicity"] == 0 else "Mixed American", axis=1)

In [ ]:
df.replace(0,"N", inplace = True)

In [ ]:
df.replace(1,"Y", inplace = True)

In [ ]:
df.head()

In [ ]:
#heatmap: I make use of an excellent library called dython. associations is a function in it
heat_map = associations(df, theil_u=True, figsize=(14, 14))

In [ ]:
# Now we can see that the relevance is high for SNP18, SNP20, SNP12, SNP11,SNP7,SNP1,SNP6,SNP8 (in descending order)

In [ ]:
# One surprising result is that SNP16, SNP15 are strongly correlated mutually (but not to ethnicity)

In [ ]:
# So we write a programme to take all these features into consideration. 
# I chose the coefficients arbitrarily (the sqare root of correlation)

In [ ]:
filename = "p_r_f1_threshold.csv" 
# I am creating a csv file in which I will temporarily store the P,R,F1 values for my 3 iterations of threshold.
# (I did a lot of manual up and down with thresholds to arrive at these values which illustrate cut-off points.)

In [ ]:
with open(filename, 'w+', newline='') as csvfile:
    csvrow = csv.writer(csvfile, delimiter=',')
    csvrow.writerow(["Threshold","TP (testing for '0')","FN","FP","TN","Precision","Recall","F1"])
    for threshold in [.12,.25,.42]:
        for (row_index, row_data) in df.iterrows():
            predicted = 0
            if(df['SNP18'][row_index] == "N"):
                predicted = predicted + 0.128761783227184
            if(df['SNP20'][row_index] == "N"):
                predicted = predicted + 0.118075965756857
            if(df['SNP12'][row_index] == "N"):
                predicted = predicted + 0.113187926182593
            if(df['SNP11'][row_index] == "Y"):
                predicted = predicted + 0.113187926182593
            if(df['SNP7'][row_index] == "Y"):
                predicted = predicted + 0.0970578459776468
            if(df['SNP1'][row_index] == "N"):
                predicted = predicted + 0.0970578459776468
            if(df['SNP6'][row_index] == "Y"):
                predicted = predicted + 0.091048330077614
            if(df['SNP8'][row_index] == "N"):
                predicted = predicted + 0.0889549851963864
            if(df['SNP4'][row_index] == "Y"):
                predicted = predicted + 0.0800359501521536
            if(df['SNP5'][row_index] == "Y"):
                predicted = predicted + 0.0726314412693259
            if predicted > threshold:
                df['Prediction'][row_index] = "Mixed American"
            else:
                df['Prediction'][row_index] = "East Asian"
        predictions = pd.crosstab(df['Ethnicity'], df['Prediction'])
        tp = predictions.iloc[0,0]
        tn = predictions.iloc[1,1]
        fn = predictions.iloc[0,1]
        fp = predictions.iloc[1,0]
        p = round(tp/(tp+fp),2)
        r = round(tp/(tp+fn),2)
        f1 = round(2*p*r/(p+r),2)
        csvrow.writerow([threshold,tp,fn,fp,tn,p,r,f1])

In [ ]:
f1_as_p_and_r_tradeoff = pd.read_csv("p_r_f1_threshold.csv")
f1_as_p_and_r_tradeoff 

## Notice how we can have trade-off between FPs and FNs.

In [ ]:
f1_as_p_and_r_tradeoff[["Precision","Recall","F1"]].plot()

## Part C-
Logistic Regression

In [ ]:
# We will again need binary number coding
df = pd.read_csv('ancestry-train.txt', sep= " ", header=None)
df.columns = ['SNP1', 'SNP2', 'SNP3', 'SNP4','SNP5', 'SNP6', 'SNP7', 'SNP8', 'SNP9', 'SNP10', 'SNP11', 'SNP12', 'SNP13', 'SNP14', 'SNP15', 'SNP16', 'SNP17', 'SNP18', 'SNP19', 'SNP20', 'Ethnicity']

In [ ]:
df['SNP20'] = df['SNP20'].apply(fix)


In [ ]:
#manual train-test split using numpy's random.rand
mask = np.random.rand(len(df)) < 0.9
train = df[mask]
test = df[~mask]
print('Train Shape: {}\nTest Shape: {}'.format(train.shape, test.shape))

In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
no_of_Mixed_Americans = train.Ethnicity.value_counts()[1]
print('There are {} Mixed American in the train data.'.format(no_of_Mixed_Americans))

In [ ]:
#randomly selecting 93 random East Asians 
# (The number 93 will change everytime you run the code in Line 45 due to the rand )
east_Asians = train[train['Ethnicity'] == 0]
mixed_Americans = train[train['Ethnicity'] == 1]
selected = east_Asians.sample(no_of_Mixed_Americans)
selected.head()

In [ ]:
#concatenating both into a subsample data set with equal class distribution
selected.reset_index(drop=True, inplace=True)
mixed_Americans.reset_index(drop=True, inplace=True)

In [ ]:
subsample = pd.concat([selected, mixed_Americans])
len(subsample)

In [ ]:
subsample = subsample.sample(frac=1).reset_index(drop=True)
subsample.head(10)

In [ ]:
# Dimensionality Reduction

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
X = subsample.drop('Ethnicity', axis=1)

In [ ]:
y = subsample['Ethnicity']

In [ ]:
#t-SNE
X_reduced_tsne = TSNE(n_components=2, random_state=42).fit_transform(X.values)

In [ ]:
# t-SNE scatter plot
import matplotlib.patches as mpatches

f, ax = plt.subplots(figsize=(24,16))


blue_patch = mpatches.Patch(color='#0A0AFF', label='East Asian')
red_patch = mpatches.Patch(color='#AF0000', label='Mixed American')

ax.scatter(X_reduced_tsne[:,0], X_reduced_tsne[:,1], c=(y == 0), cmap='coolwarm', label='East Asian', linewidths=2)
ax.scatter(X_reduced_tsne[:,0], X_reduced_tsne[:,1], c=(y == 1), cmap='coolwarm', label='Mixed American', linewidths=2)
ax.set_title('t-SNE', fontsize=14)

ax.grid(True)

ax.legend(handles=[blue_patch, red_patch])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=42)

In [ ]:
X_train = X_train.values
X_validation = X_test.values
y_train = y_train.values
y_validation = y_test.values

In [ ]:
print('X_shapes:\n', 'X_train:', 'X_validation:\n', X_train.shape, X_validation.shape, '\n')
print('Y_shapes:\n', 'Y_train:', 'Y_validation:\n', y_train.shape, y_validation.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
kfold = KFold(n_splits=10, random_state=42)
cv_results = cross_val_score(LogisticRegression(), X_train, y_train, cv=kfold, scoring='roc_auc')
name = "LR"
msg = '%s: %f (%f)' % (name, cv_results.mean(), cv_results.std())
print(msg)

In [ ]:
logisticRegr = LogisticRegression()
logreg = logisticRegr.fit(X_train, y_train) #fitting the model with data

In [ ]:
np.exp(logreg.coef_) #These are the coefficients for the features

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
#evaluating
cnf_matrix = confusion_matrix(y_test,y_pred)
cnf_matrix

In [ ]:
precision = cnf_matrix[0][0]/(cnf_matrix[0][0]+cnf_matrix[1][0])
recall = cnf_matrix[0][0]/(cnf_matrix[0][0]+cnf_matrix[0][1])
f1 = 2*precision*recall/(precision+recall)
print(f"Precision = {round(precision,2)}, Recall = {round(recall,2)}, F1 = {round(f1,2)}")

## Comparison
Though it may depend on the random samples that you got, I found the accuracy of the LogisticRegression model better everytime I ran the code.